In [7]:
import boto3

kendra_client = boto3.client("kendra")
translate_client = boto3.client('translate')

def QueryKendra(index_id,query): 
    response = kendra_client.retrieve(
        QueryText = query,
        IndexId = index_id)
    return response

In [24]:
data_source_id = ""

In [26]:
response_data = kendra_client.describe_data_source(
    Id=data_source_id,
    IndexId = index_id
)

In [33]:
TargetLanguage = get_target_language_code(data_source_id,index_id)

In [32]:
def get_target_language_code(data_source_id,index_id): 
    response_data_source = kendra_client.describe_data_source(
        Id = data_source_id,
        IndexId = index_id
    )
    return response_data_source['LanguageCode']

In [34]:
TargetLanguage

'en'

In [6]:
def TranslateText(text,SourceLanguageCode,TargetLanguage):
    response = translate_client.translate_text(
    Text=text,
    SourceLanguageCode=SourceLanguageCode,
    TargetLanguageCode=TargetLanguage
)
    translated_text = response['TranslatedText']
    source_language_code = response['SourceLanguageCode'] #you need SourceLanguageCode to answer in the original language
    return translated_text, source_language_code

In [30]:
text = "¿que es Amazon S3?"
index_id = ""

query,source_language_code = TranslateText(text,"auto",TargetLanguage)
response = QueryKendra(index_id,query)

#print the result

for query_result in response["ResultItems"]:
    print("-------------------")
    document_title = query_result['DocumentTitle']
    document_title_translated,language = TranslateText(document_title,TargetLanguage,source_language_code)
    print("DocumentTitle: " + document_title_translated)
    document_content = query_result['Content']
    document_content_translated,language = TranslateText(document_content,TargetLanguage,source_language_code)
    print("Content: ",document_content_translated)
    print("Go deeper: ", query_result['DocumentURI']) 


-------------------
DocumentTitle: Introducción a Amazon S3 - Amazon Simple Storage Service
Content:  Documentación de AWS Guía para desarrolladores de Amazon Simple Storage Service (S3) Descripción general de Amazon S3 y esta guíaVentajas de usar Amazon S3 Conceptos de Amazon S3 Características de Amazon S3 Interfaces de programación de aplicaciones (API) de Amazon S3 Pago de los servicios relacionados con Amazon S3 Introducción a Amazon S3 Esta introducción a Amazon Simple Storage Service (Amazon S3) proporciona un resumen detallado de este servicio web. Después de leer esta sección, deberías tener una buena idea de lo que ofrece y cómo puede encajar en tu negocio. Temas Descripción general de Amazon S3 y esta guía Ventajas de usar Amazon S3 Conceptos de Amazon S3 Amazon S3 presenta las interfaces de programación de aplicaciones (API) de Amazon S3 Pago de los servicios relacionados con Amazon S3 Descripción general de Después de leer esta sección, tendrá una buena idea de lo que ofre

In [10]:
import json
newline, bold, unbold = "\n", "\033[1m", "\033[0m"
num_return_sequences = 3
parameters = {
    "max_length": 50,
    "max_time": 50,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}

In [15]:
endpoint_name = 'jumpstart-example-huggingface-text2text-2023-08-09-21-40-22-962'

In [11]:
def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response

In [12]:
def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text

In [17]:
def summarization(text,query):
    
    payload = {"text_inputs": f"{text}\n\nBased on the above article, answer a question. {query}", **parameters}
    query_response = query_endpoint_with_json_payload(
            json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
        )

    generated_texts = parse_response_multiple_texts(query_response)

    print(f"{bold} The {num_return_sequences} summarized results are{unbold}:{newline}")

    for idx, each_generated_text in enumerate(generated_texts):
        #Translate the answer to the original language of the question
        answer_text_translated,language = TranslateText(each_generated_text,TargetLanguage,source_language_code)

        print(f"{bold}Result {idx}{unbold}: {answer_text_translated}{newline}")  

    return

In [23]:
text = "¿que es Amazon s3?"
target_language_code = "en" #English language code
query,source_language_code = TranslateText(text,"auto",target_language_code)
response = QueryKendra(index_id,query)

for query_result in response["ResultItems"]:
    print("-------------------")
    document_title = query_result['DocumentTitle']
    document_title_translated,language = TranslateText(document_title,TargetLanguage,source_language_code)
    print("DocumentTitle: " + document_title_translated)
    document_content = query_result['Content']
    document_content_translated,language = TranslateText(document_content,TargetLanguage,source_language_code)
    print("Go deeper: ", query_result['DocumentURI'])
    summarization(document_content,query)


-------------------
DocumentTitle: Introducción a Amazon S3 - Amazon Simple Storage Service
Go deeper:  http://docs.aws.amazon.com/AmazonS3/latest/dev/Introduction.html
 The 3 summarized results are:

Result 0: un servicio web que puede utilizar para almacenar y recuperar cualquier cantidad de datos, en cualquier momento y desde cualquier lugar de la web

Result 1: d).

Result 2: (iv).

-------------------
DocumentTitle: Claves de condición, recursos y acciones para Amazon S3 - Amazon Simple Storage Service
Go deeper:  http://docs.aws.amazon.com/AmazonS3/latest/dev/list_amazons3.html
 The 3 summarized results are:

Result 0: S3 es un servicio de almacenamiento en la nube que permite a los usuarios almacenar, recuperar y administrar archivos.

Result 1: Servicio de almacenamiento

Result 2: S3 es un servicio de almacenamiento que permite a los usuarios almacenar, recuperar y administrar archivos y carpetas en la nube.

-------------------
DocumentTitle: Claves de condición, recursos y a